In [1]:
from dataclasses import dataclass
from typing import Any


In [2]:
@dataclass
class Number:
    value: Any
    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return f'«#{self}»'
    
    def reducible(self):
        return False
    

In [3]:
n = Number(12)

In [4]:
@dataclass
class Add:
    left: Any
    right: Any
    
    def __str__(self):
        return  f'{self.left} + {self.right}'
    
    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return True
    
    def reduce(self):
        if self.left.reducible():
            return Add(self.left.reduce(), self.right)
        elif self.right.reducible():
            return Add(self.left, self.right.reduce())
        
        return Number(self.left.value + self.right.value)
    
    
    
    

In [5]:
a = Number(1) 
b = Number(2)

Add(a, b)

«1 + 2»

In [6]:
@dataclass
class Multiply:
    left: Any
    right: Any
    
    def __str__(self):
        return  f'{self.left} * {self.right}'
    
    def __repr__(self):
        return f'«{self}»'

    def reducible(self):
        return True
    
    def reduce(self):
        if self.left.reducible():
            return Multiply(self.left.reduce(), self.right)
        elif self.right.reducible():
            return Multiply(self.left, self.right.reduce())
        
        return Number(self.left.value * self.right.value)


In [7]:
a = Number(1) 
b = Number(2)

Multiply(a, b)

«1 * 2»

In [8]:
Add(
  Multiply(Number(1), Number(2)),
  Multiply(Number(3), Number(4))
)

«1 * 2 + 3 * 4»

In [9]:
expression = Add(
  Multiply(Number(1), Number(2)),
  Multiply(Number(3), Number(4))
)

In [10]:
expression.reduce()

«2 + 3 * 4»

In [11]:
@dataclass
class Machine:
    expression: Any
    
    def step(self):
        self.expression = self.expression.reduce()
    
    def run(self):
        
        while self.expression.reducible():
            print(self.expression)
            self.step()
        
        print(self.expression)


    

In [12]:
expression = Add(
  Multiply(Number(1), Number(2)),
  Multiply(Number(3), Number(4))
)

In [13]:
machine = Machine(expression)

In [14]:
machine.run()

1 * 2 + 3 * 4
2 + 3 * 4
2 + 12
14


In [15]:
@dataclass
class Boolean:
    value: Any
    
    def __str__(self):
        return str(self.value)

    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return False
    

In [16]:
@dataclass
class LessThan:
    left: Any
    right: Any
    
    def __str__(self):    
        return f'{self.left} < {self.right}'
    
    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return True
    
    def reduce(self):
        if self.left.reducible():
            return LessThan(self.left.reduce(), self.right)
        elif self.right.reducible():
            return LessThan(self.left, self.right.reduce())
        return Boolean(self.left.value < self.right.value)
    
    

In [17]:
machine = Machine(LessThan(Number(5), Add(Number(1), Number(2))))

In [18]:
machine.run()

5 < 1 + 2
5 < 3
False


In [19]:
@dataclass
class Variable:
    name: Any
    
    def __str__(self):
        return str(self.name)

    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return True
    
    def reduce(self, environment):
        return environment[self.name]
    
    

In [20]:
@dataclass
class Add:
    left: Any
    right: Any
    
    def __str__(self):
        return  f'{self.left} + {self.right}'
    
    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return True
    
    def reduce(self, environment):
        if self.left.reducible():
            return Add(self.left.reduce(environment), self.right)
        elif self.right.reducible():
            return Add(self.left, self.right.reduce(environment))
        
        return Number(self.left.value + self.right.value)

In [21]:
@dataclass
class Multiply:
    left: Any
    right: Any
    
    def __str__(self):
        return  f'{self.left} * {self.right}'
    
    def __repr__(self):
        return f'«{self}»'

    def reducible(self):
        return True
    
    def reduce(self, environment):
        if self.left.reducible():
            return Multiply(self.left.reduce(environment), self.right)
        elif self.right.reducible():
            return Multiply(self.left, self.right.reduce(environment))
        
        return Number(self.left.value * self.right.value)



In [22]:
@dataclass
class LessThan:
    left: Any
    right: Any
    
    def __str__(self):    
        return f'{self.left} < {self.right}'
    
    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return True
    
    def reduce(self):
        if self.left.reducible():
            return LessThan(self.left.reduce(environment), self.right)
        elif self.right.reducible():
            return LessThan(self.left, self.right.reduce(environment))
        return Boolean(self.left.value < self.right.value)

In [23]:
@dataclass
class Machine:
    expression: Any
    environment: Any
    
    def step(self):
        self.expression = self.expression.reduce(self.environment)
    
    def run(self):
        
        while self.expression.reducible():
            print(self.expression)
            self.step()
        
        print(self.expression)

In [24]:
machine = Machine(Add(Variable("x"), Variable("y")), {"x": Number(3), "y": Number(4)})

In [25]:
machine.run()

x + y
3 + y
3 + 4
7


In [26]:
@dataclass
class DoNothing:
    def __str__(self):
        return 'do-nothing'
   
    def __repr__(self):
        return f'«{self}»'
    
    def __eq__(self):
        return isinstance(self,DoNothing)
    
    def reducible(self):
        return False
    

In [27]:
@dataclass
class Assign:
    name: Any
    expression: Any
    
    def __str__(self):
        return f'{self.name} = {self.expression}'
   
    def __repr__(self):
        return f'«{self}»'
    
    def reducible(self):
        return True
    
    def reduce(self, environment):
        if self.expression.reducible():
            return [Assign(self.name, self.expression.reduce(environment)), environment]
        environment.update({self.name: self.expression})
        return [DoNothing(), environment]
    

    

In [28]:
statement = Assign("x", Add(Variable("x"), Number(1)))


In [29]:
environment = {"x": Number(2)}

In [30]:
statement, environment = statement.reduce(environment)

In [31]:
statement, environment = statement.reduce(environment)

In [32]:
statement, environment = statement.reduce(environment)
statement

«do-nothing»

In [33]:
print(environment)

{'x': «#3»}


In [38]:
@dataclass
class Machine:
    expression: Any
    environment: Any
    
    def step(self):
        self.expression,self.environment = self.expression.reduce(self.environment)
    
    def run(self):
        
        while self.expression.reducible():
            print(self.expression, self.environment)
            self.step()
        
        print(self.expression,self.environment)
        

In [40]:
Machine(Assign("x", Add(Variable("x"), Number(1))), {"x": Number(2)}).run()

x = x + 1 {'x': «#2»}
x = 2 + 1 {'x': «#2»}
x = 3 {'x': «#2»}
do-nothing {'x': «#3»}


In [ ]:
@dataclass
class If:
    condition: Any
    consequence: Any
    alternative: Any
    
    def __str__(self):
        return f"if ({self.condition}) {{  {self.consequence}  }} else {{  {self.alternative  }}"
        
    def __repr__(self):
        return f'«{self}»'  
    
    